#Semantizisest time test

Make sure semanticizest is installed

https://github.com/semanticize/semanticizest

First let's test semanticizest itself, using code from: http://semanticizest.readthedocs.org/en/master/

In [1]:
import re
from semanticizest import Semanticizer

In [2]:
sem = Semanticizer('/home/carlosm/Software/semanticizest/nl.model')

In [3]:
text = ("Roken... je laatste... Hij kijkt schichtig achterom, alsof het gevaar meer van die kant dreigt dan van mij. " 
        "Ik ga hem in zijn gezicht schieten, flitst het door mij heen. Ik ga dat onschuldige, nog bijna baardloze " 
        "gezicht openrijten. Ik glimlach hem toe en voel de tranen in mijn ogen branden.")
toks = re.findall('\w+', text)

for idx0,idx1,link,p_link in sem.all_candidates(toks):
    if p_link>0.5:
        print link,'%2.2f %%'%(p_link*100),toks[idx0:idx1]

Roken (tabak) 53.19 % ['Roken']
Jij 100.00 % ['je']
Hij (single) 100.00 % ['Hij']
Entiteit 66.67 % ['het']
Gevaar (risico) 100.00 % ['gevaar']
Bestelwagen 66.67 % ['van']
Kant (textiel) 56.38 % ['kant']
Dan (graad) 74.17 % ['dan']
Bestelwagen 66.67 % ['van']
Entiteit 66.67 % ['het']
Heen 100.00 % ['heen']
Digital audio tape 87.50 % ['dat']
Lachen (gedrag) 100.00 % ['glimlach']
Duits 100.00 % ['de']
Traan (oog) 69.35 % ['tranen']


# Time trials
Now lets load full books and take a look at how long it takes to semanticize a full book.

In [4]:
import BeautifulSoup
from collections import defaultdict

In [5]:
def semanticizeBook(bookXML):
    soup = BeautifulSoup.BeautifulSoup(open(bookXML))
    paragraphs = soup.findAll(['p','pb'])

    pages = []

    currPage = defaultdict(lambda: 0)
    pages.append(currPage)

    for p in paragraphs:
        if p.name=='pb':
            # New Page
            currPage = defaultdict(lambda: 0)
            pages.append(currPage)

        toks = re.findall('\w+', p.text)

        for idx0,idx1,link,p_link in sem.all_candidates(toks):
            if p_link>0.5:
                currPage[link] += 1
    return pages


In [7]:
%%timeit

# Timeit one book from Karina corpus
bookXML = "../data/Dutch_literature_Karina/corpusHuygens/OR_Ferron_Stierenoffer_1975_pags.xml"
semanticizeBook(bookXML)

1 loops, best of 3: 999 ms per loop


Now lets have a look at all books of the Karina corpus and find the average time per book

In [13]:
import glob
import timeit
import numpy as np

In [10]:
books = glob.glob("../data/Dutch_literature_Karina/corpusHuygens/*.xml")

In [11]:
times = []
nRepeats = 10

for book in books:
    t = timeit.Timer("semanticizeBook('" + book + "')", "from __main__ import semanticizeBook")
    times.append(t.timeit(number=nRepeats))

In [14]:
print 'Average time per book: %2.4f seconds per book'%(np.array(times)/nRepeats).mean()

Average time per book: 1.2180 seconds per book
